In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import shutil
import zipfile

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
shutil.copyfile("./gdrive/My Drive/data/wrs.zip", "./wrs.zip")
with zipfile.ZipFile('./wrs.zip') as existing_zip:
  existing_zip.extractall('./')
shutil.copyfile("./gdrive/My Drive/data/param5_700.hdf5", "./params.hdf5")

'./params.hdf5'

In [3]:
import keras
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Activation, GlobalAveragePooling2D, Flatten, Input
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet import MobileNet
import numpy as np
import pandas as pd
import io
import random
import pickle
import cv2
import os
import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import time


Using TensorFlow backend.


In [0]:
input_shape = (300, 300, 3)
classes = ['standing' , 'lying', 'nothing']
NUM_CLASSES = len(classes)
in_dirs = classes
in_images = []
x_train = []
y_train = []
x_test = []
y_test = []
all_test_x = []
all_test_y = []
rate = 0.1
file_num =0

classes_num = [0] * len(classes)

In [5]:
print(NUM_CLASSES)

3


## ファイルを数えている

In [6]:
# table = []
for images in tqdm.tqdm(in_dirs):
#   all_images_x = []
  images_list = os.listdir(images)
#   i = 0
#   j = 0
#   table
#   local_table
  for image_path in images_list:
    file_num += 1
    classes_num[in_dirs.index(images)] += 1
#     image = cv2.imread(images + '/' + image_path)
#     image = cv2.resize(image,(50, 50))
#     if i == 0:
#       local_table = image
# #       print(type(local_table))
# #       print(type(image))
#     else:
#       local_table = cv2.hconcat([local_table, image])
#     if i == 5:
#       i = -1
#       if j == 0:
#         table = local_table
#       else:
#         table = cv2.vconcat([table, local_table])
#       local_table = []
#       j+=1
#       if j == 5:
#         plt.imshow(table)
#         plt.show()
#         time.sleep(0.1)
#     i += 1
   


  0%|          | 0/3 [00:00<?, ?it/s]


FileNotFoundError: ignored

In [96]:
print(classes_num)

[1109, 3634, 5211]


### vgg16

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
x = base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='relu')(x)
prediction=Dense(NUM_CLASSES, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=prediction)
for layer in base_model.layers[:15]:
    layer.trainable=False

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.load_weights('./params.hdf5')
# model.summary()

###

In [0]:
def Train(x, y, epoch=20, batch_size = 128):
  history = model.fit(x, y, batch_size=batch_size, epochs=epoch, verbose=1, validation_split=0.2)

In [0]:
def Read(start, end):
  all_x = []
  all_y = []
  global x_train
  global y_train
  x_train = []
  y_train = []
  global x_test
  global y_test
  for images in in_dirs:
    images_list = os.listdir(images)
    count = 0
    for image_path in images_list:
      count+=1
      if count < start:
        continue
      if count >= end:
        break
      image = cv2.imread(images + '/' + image_path)
      if image is None:
        print("no image!")
        continue
      image = cv2.resize(image, (300, 300))
      all_x.append(image)
      y = [0] * NUM_CLASSES
      y[in_dirs.index(images)] = 1
      all_y.append(y)
  temp_x = []
  temp_y = []
  (x_train, temp_x, y_train, temp_y) = train_test_split(all_x, all_y, test_size = 0, shuffle=True)
  
  if type(temp_x) is not list:
    temp_x = temp_x.tolist()
  if type(temp_y) is not list:
    temp_y = temp_y.tolist()
  x_test.append(temp_x)
  y_test.append(temp_y)
  
#   x_test = np.array(x_test, dtype='float32') / 255
#   y_test = np.array(y_test)
  del all_x, all_y

In [72]:
x_train = []
y_train = []
x_test = []
y_test = []
all_test_x = []
all_test_y = []
print("file num is " + str(file_num))
distance = 100
for i in range(distance * 0, int(file_num * 0.8 / 4), distance):
  print(i * 4)
  Read(i, i + distance)
  for x in x_test:    
    all_test_x.append(x)
  for y in y_test:
    all_test_y.append(y)
  x_train = np.array(x_train, dtype='float32') / 255
  y_train = np.array(y_train)
  print(x_train.shape)
#   print(y_train)
  Train(x_train, y_train, 5, 64)
  open('./gdrive/My Drive/data/model5_' + str(i) + '.json', 'w').write(model.to_json())
  model.save_weights('./gdrive/My Drive/data/param5_' + str(i) + '.hdf5')
  

file num is 3681
0
(237, 300, 300, 3)
Train on 189 samples, validate on 48 samples
Epoch 1/5
189/189 [==============================] - 7s 39ms/step - loss: 1.4722 - categorical_accuracy: 0.3545 - val_loss: 1.0901 - val_categorical_accuracy: 0.3750
Epoch 2/5
189/189 [==============================] - 4s 23ms/step - loss: 1.0888 - categorical_accuracy: 0.3651 - val_loss: 1.0931 - val_categorical_accuracy: 0.2708
Epoch 3/5
189/189 [==============================] - 4s 23ms/step - loss: 1.1793 - categorical_accuracy: 0.3175 - val_loss: 1.0233 - val_categorical_accuracy: 0.5417
Epoch 4/5
189/189 [==============================] - 4s 23ms/step - loss: 1.1703 - categorical_accuracy: 0.3492 - val_loss: 0.9494 - val_categorical_accuracy: 0.7083
Epoch 5/5
189/189 [==============================] - 4s 23ms/step - loss: 1.0002 - categorical_accuracy: 0.5556 - val_loss: 0.9437 - val_categorical_accuracy: 0.5625
400
(240, 300, 300, 3)
Train on 192 samples, validate on 48 samples
Epoch 1/5
192/192 [

In [86]:
# from google.colab import files
# files = files.upload()
# all_test_x = np.array([cv2.resize(cv2.imread('./image.jpg'),(300, 300))])
# print(all_test_x)
# print(all_test_x)

# for x in all_test_x:
#   print(x)
#   if type(x) is not list:
#     x = x.tolist()
#   test_x.append(x)
# for y in all_test_y:
#   if type(y) is not list:
#     y = y.tolist()
#   test_y.append(y)
# print(len(y_test))
# print(len(x_test))

np.set_printoptions(suppress=True)
(trainx,all_test_x,trainy, all_test_y) = train_test_split(x_test, y_test, test_size = 1, shuffle=1)
all_test_x = np.array(all_test_x, dtype='float32') / 255
all_test_x = all_test_x[0]
# print(len(all_test_x))
# print(all_test_x)
for num in range(len(all_test_x)):
  result = model.predict(np.array([all_test_x[num]]))
  print(result)
  print(all_test_y[0][num])
  print()

[[0. 1. 0.]]
[0, 1, 0]

[[0.9999901  0.00000988 0.        ]]
[1, 0, 0]

[[0. 1. 0.]]
[0, 1, 0]

[[0.0019906  0.02004933 0.97796005]]
[0, 0, 1]

[[1. 0. 0.]]
[1, 0, 0]

[[0.00023565 0.00529395 0.99447036]]
[0, 0, 1]

[[0. 1. 0.]]
[0, 1, 0]

[[0. 1. 0.]]
[0, 1, 0]

[[1. 0. 0.]]
[1, 0, 0]

[[1. 0. 0.]]
[1, 0, 0]

[[0.0000001  0.99999964 0.00000025]]
[0, 1, 0]

[[0.9998387  0.00016021 0.00000109]]
[1, 0, 0]

[[0.00024014 0.00526565 0.99449414]]
[0, 0, 1]

[[0.00065476 0.00984888 0.9894964 ]]
[0, 0, 1]

[[0. 1. 0.]]
[0, 1, 0]

[[0. 1. 0.]]
[0, 1, 0]

[[0.0039007  0.03129405 0.96480525]]
[0, 0, 1]

[[1. 0. 0.]]
[1, 0, 0]

[[1. 0. 0.]]
[1, 0, 0]

[[0.0004542 0.009259  0.9902868]]
[0, 0, 1]

[[1. 0. 0.]]
[1, 0, 0]

[[0.00099947 0.01325588 0.98574466]]
[0, 0, 1]

[[1.         0.00000003 0.        ]]
[1, 0, 0]

[[0.00000002 1.         0.00000002]]
[0, 1, 0]

[[1. 0. 0.]]
[1, 0, 0]

[[0. 1. 0.]]
[0, 1, 0]

[[0. 1. 0.]]
[0, 1, 0]

[[1. 0. 0.]]
[1, 0, 0]

[[1. 0. 0.]]
[1, 0, 0]

[[0. 1. 0.]]
[0, 1,

# ゴミ箱

In [0]:
history = model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=1, validation_split=0.1)

In [0]:
np.set_printoptions(suppress=True)
result = model.predict(x_test);
for i in range(len(result)):
  print(result[i])
  print(y_test[i])

In [0]:
open('./gdrive/My Drive/data/model.json', 'w').write(model.to_json())
model.save_weights('./gdrive/My Drive/data/param.hdf5')

In [0]:
model = Sequential()

model.add(Conv2D(64,(3,3),padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=((2,2))))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.load_weights('params.hdf5')
model.summary()

In [0]:
np.set_printoptions(suppress=True)
result = model.predict(x_test);
for i in range(len(result)):
  print(result[i])
  print(y_test[i])